## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
########################### 1-1. recursive model by store ##########################
####################################################################################

In [4]:
ver, KKK = 'priv', 0
CLUSTERS_IDS = [0,1,2,3,4,5,6,7,8,9]

In [5]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [6]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [7]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_cluster(cluster):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[df['tskm_10']==cluster]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3

    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for cluster_id in CLUSTERS_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_cluster10_'+cluster_id+'.pkl')
        temp_df['cluster_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [8]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                } 


In [9]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id', 'dept_id', 'cat_id', 'store_id', 'state_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [10]:
########################### Train Models
#################################################################################
for cluster_id in CLUSTERS_IDS:
    print('Train cluster_', cluster_id)
    
    grid_df, features_columns = get_data_by_cluster(cluster_id)
    
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)
    
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    
    d_sales = grid_df[['d','sales']]
    substitute = d_sales['sales'].values
    substitute[(d_sales['d'] > END_TRAIN)] = np.nan
    grid_df['sales'] = substitute
    
    grid_df.to_pickle(processed_data_dir+'test_cluster10_'+str(cluster_id)+'.pkl')
    del grid_df, d_sales, substitute
    

    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    display(pd.DataFrame({'name':estimator.feature_name(),
                          'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))

    
    model_name = model_dir+'lgb_model_cluster10_'+str(cluster_id)+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    del train_data, valid_data, estimator
    gc.collect()
    
    MODEL_FEATURES = features_columns

Train cluster_ 0
[100]	valid_0's rmse: 0.838565
[200]	valid_0's rmse: 0.811489
[300]	valid_0's rmse: 0.800537
[400]	valid_0's rmse: 0.792201
[500]	valid_0's rmse: 0.786886
[600]	valid_0's rmse: 0.784342
[700]	valid_0's rmse: 0.782296
[800]	valid_0's rmse: 0.780434
[900]	valid_0's rmse: 0.778774
[1000]	valid_0's rmse: 0.777057
[1100]	valid_0's rmse: 0.775415
[1200]	valid_0's rmse: 0.773952
[1300]	valid_0's rmse: 0.772375
[1400]	valid_0's rmse: 0.771146
[1500]	valid_0's rmse: 0.769849
[1600]	valid_0's rmse: 0.768558
[1700]	valid_0's rmse: 0.767227
[1800]	valid_0's rmse: 0.766062
[1900]	valid_0's rmse: 0.7649
[2000]	valid_0's rmse: 0.763452
[2100]	valid_0's rmse: 0.762273
[2200]	valid_0's rmse: 0.761012
[2300]	valid_0's rmse: 0.759845
[2400]	valid_0's rmse: 0.758584
[2500]	valid_0's rmse: 0.757264
[2600]	valid_0's rmse: 0.755921
[2700]	valid_0's rmse: 0.754821
[2800]	valid_0's rmse: 0.753637
[2900]	valid_0's rmse: 0.752424
[3000]	valid_0's rmse: 0.751369


,name,imp
0,item_id,435472
53,rolling_mean_180,115011
22,tm_w,110793
54,rolling_std_180,110734
52,rolling_std_60,91502
21,tm_d,91079
50,rolling_std_30,75950
10,item_nunique,75880
29,enc_item_id_store_id_std,73469
28,enc_item_id_store_id_mean,73317


Train cluster_ 1
[100]	valid_0's rmse: 2.74435
[200]	valid_0's rmse: 2.41648
[300]	valid_0's rmse: 2.37839
[400]	valid_0's rmse: 2.36192
[500]	valid_0's rmse: 2.35222
[600]	valid_0's rmse: 2.3451
[700]	valid_0's rmse: 2.33873
[800]	valid_0's rmse: 2.3334
[900]	valid_0's rmse: 2.32802
[1000]	valid_0's rmse: 2.32299
[1100]	valid_0's rmse: 2.3183
[1200]	valid_0's rmse: 2.31383
[1300]	valid_0's rmse: 2.30973
[1400]	valid_0's rmse: 2.30582
[1500]	valid_0's rmse: 2.30177
[1600]	valid_0's rmse: 2.29852
[1700]	valid_0's rmse: 2.29469
[1800]	valid_0's rmse: 2.29098
[1900]	valid_0's rmse: 2.28739
[2000]	valid_0's rmse: 2.28285
[2100]	valid_0's rmse: 2.27926
[2200]	valid_0's rmse: 2.27551
[2300]	valid_0's rmse: 2.27224
[2400]	valid_0's rmse: 2.26925
[2500]	valid_0's rmse: 2.26556
[2600]	valid_0's rmse: 2.26185
[2700]	valid_0's rmse: 2.2584
[2800]	valid_0's rmse: 2.25482
[2900]	valid_0's rmse: 2.25168
[3000]	valid_0's rmse: 2.24837


,name,imp
0,item_id,264992
22,tm_w,47923
53,rolling_mean_180,43904
54,rolling_std_180,43586
21,tm_d,43479
55,rolling_mean_tmp_1_7,42771
52,rolling_std_60,36717
10,item_nunique,34127
50,rolling_std_30,34062
46,rolling_std_7,31432


Train cluster_ 2
[100]	valid_0's rmse: 1.90251
[200]	valid_0's rmse: 1.81014
[300]	valid_0's rmse: 1.79134
[400]	valid_0's rmse: 1.78091
[500]	valid_0's rmse: 1.77343
[600]	valid_0's rmse: 1.76747
[700]	valid_0's rmse: 1.76303
[800]	valid_0's rmse: 1.75896
[900]	valid_0's rmse: 1.75506
[1000]	valid_0's rmse: 1.75135
[1100]	valid_0's rmse: 1.74783
[1200]	valid_0's rmse: 1.74417
[1300]	valid_0's rmse: 1.74093
[1400]	valid_0's rmse: 1.73805
[1500]	valid_0's rmse: 1.73501
[1600]	valid_0's rmse: 1.73205
[1700]	valid_0's rmse: 1.72914
[1800]	valid_0's rmse: 1.72634
[1900]	valid_0's rmse: 1.72349
[2000]	valid_0's rmse: 1.72031
[2100]	valid_0's rmse: 1.71742
[2200]	valid_0's rmse: 1.71476
[2300]	valid_0's rmse: 1.71214
[2400]	valid_0's rmse: 1.70945
[2500]	valid_0's rmse: 1.70667
[2600]	valid_0's rmse: 1.70396
[2700]	valid_0's rmse: 1.70122
[2800]	valid_0's rmse: 1.69829
[2900]	valid_0's rmse: 1.69563
[3000]	valid_0's rmse: 1.69339


...

,name,imp
0,item_id,399663
53,rolling_mean_180,86534
54,rolling_std_180,85446
22,tm_w,82969
21,tm_d,74213
52,rolling_std_60,73718
50,rolling_std_30,68063
51,rolling_mean_60,61280
10,item_nunique,58780
48,rolling_std_14,58401


Train cluster_ 8
[100]	valid_0's rmse: 3.90367
[200]	valid_0's rmse: 3.18998
[300]	valid_0's rmse: 3.01903
[400]	valid_0's rmse: 2.95686
[500]	valid_0's rmse: 2.92857
[600]	valid_0's rmse: 2.91902
[700]	valid_0's rmse: 2.9123
[800]	valid_0's rmse: 2.90519
[900]	valid_0's rmse: 2.89699
[1000]	valid_0's rmse: 2.89165
[1100]	valid_0's rmse: 2.88204
[1200]	valid_0's rmse: 2.87741
[1300]	valid_0's rmse: 2.86901
[1400]	valid_0's rmse: 2.86459
[1500]	valid_0's rmse: 2.85771
[1600]	valid_0's rmse: 2.85178
[1700]	valid_0's rmse: 2.84632
[1800]	valid_0's rmse: 2.84117
[1900]	valid_0's rmse: 2.83437
[2000]	valid_0's rmse: 2.82926
[2100]	valid_0's rmse: 2.82616
[2200]	valid_0's rmse: 2.82171
[2300]	valid_0's rmse: 2.81647
[2400]	valid_0's rmse: 2.81271
[2500]	valid_0's rmse: 2.80777
[2600]	valid_0's rmse: 2.80578
[2700]	valid_0's rmse: 2.80244
[2800]	valid_0's rmse: 2.79771
[2900]	valid_0's rmse: 2.78891
[3000]	valid_0's rmse: 2.78612


,name,imp
0,item_id,36487
55,rolling_mean_tmp_1_7,5252
21,tm_d,5160
22,tm_w,4885
53,rolling_mean_180,4500
54,rolling_std_180,3988
59,rolling_mean_tmp_7_7,3448
56,rolling_mean_tmp_1_14,3406
26,tm_dw,3345
46,rolling_std_7,3148


Train cluster_ 9
[100]	valid_0's rmse: 2.03402
[200]	valid_0's rmse: 1.89748
[300]	valid_0's rmse: 1.87732
[400]	valid_0's rmse: 1.86719
[500]	valid_0's rmse: 1.8589
[600]	valid_0's rmse: 1.85308
[700]	valid_0's rmse: 1.84802
[800]	valid_0's rmse: 1.84293
[900]	valid_0's rmse: 1.83865
[1000]	valid_0's rmse: 1.83455
[1100]	valid_0's rmse: 1.83048
[1200]	valid_0's rmse: 1.82655
[1300]	valid_0's rmse: 1.82249
[1400]	valid_0's rmse: 1.81908
[1500]	valid_0's rmse: 1.81607
[1600]	valid_0's rmse: 1.81284
[1700]	valid_0's rmse: 1.80949
[1800]	valid_0's rmse: 1.80646
[1900]	valid_0's rmse: 1.80371
[2000]	valid_0's rmse: 1.80093
[2100]	valid_0's rmse: 1.79812
[2200]	valid_0's rmse: 1.79547
[2300]	valid_0's rmse: 1.79294
[2400]	valid_0's rmse: 1.79018
[2500]	valid_0's rmse: 1.78736
[2600]	valid_0's rmse: 1.78456
[2700]	valid_0's rmse: 1.78235
[2800]	valid_0's rmse: 1.77957
[2900]	valid_0's rmse: 1.77709
[3000]	valid_0's rmse: 1.7747


,name,imp
0,item_id,447093
22,tm_w,90840
53,rolling_mean_180,84264
54,rolling_std_180,82617
21,tm_d,77747
55,rolling_mean_tmp_1_7,70145
52,rolling_std_60,68349
50,rolling_std_30,64941
10,item_nunique,64237
48,rolling_std_14,60384
